In [1]:
# Scraping related imports
from bs4 import BeautifulSoup
import requests
import json
import re
import time

In [2]:
# Funcion parses Genre info
genre_template = re.compile(r"[\w][-&/ \w]*")

def parse_genre(td):
    for s in td.strings:
        g = genre_template.match(s)
        if g: return g[0]
    return None

In [15]:
# Funcion parses Origin info
origin_template = re.compile(r"[\w, ]")

def parse_origin(td):
    origin = ""
    for s in td.strings:
        if origin_template.match(s):
            origin += s
    if len(origin):
        return origin
    else:
        return None

In [5]:
# Funcion parses Birth info (date and place)
def parse_born(td):
    # Handle birth date
    bday = None
    span = td.find('span', class_='bday')
    if span:
        bday = str(span.string)

    # Handle Birth place
    bplace = ""
    td_brs = td.find_all('br') # May be 2 <br> - 1st after birth name,
    if td_brs:                 # 2nd after birth date. Birth name may be missing, so only 1 <br> is possible
        td_br = td_brs[-1]     # We look for the last <br>
    else:
        td_br = None
        
    if td_br and td_br.next_siblings:
        for sib in td_br.next_siblings:
            try:
                for s in sib.strings:
                    if origin_template.match(s):
                        bplace += s
            except:
                if origin_template.match(sib):
                    bplace += sib

    if len(bplace) == 0:
        bplace = None # We need NULL in the table, and "" isn't NULL, only None is NULL
    
    return bday, bplace

In [6]:
session = requests.Session()

In [13]:
url = 'https://en.wikipedia.org/wiki/Original_Concept'
response = session.get(url)
soup = BeautifulSoup(response.text, 'lxml')
artist_table=soup.find('table', class_='infobox')

In [18]:
headers = artist_table.find_all('th')
td = None
is_band = None
for th in headers:
    if re.match("Genre", str(th.string), re.I):
        genre = parse_genre(th.next_sibling)
        print("Genre:", genre)
        is_band = 0
    elif re.match("Born", str(th.string), re.I):
        dob, pob = parse_born(th.next_sibling)
        print("DOB:", dob, " POB:", pob)
        if dob: is_band = 0
    elif re.match("Origin$", str(th.string), re.I):
        origin = parse_origin(th.next_sibling)
        print("Origin:", origin)
    elif re.match("Member", str(th.string), re.I):
        print("is band")
        is_band = 1
print(f"is_band={is_band}")

<th scope="row">Origin</th>
<td><a class="mw-redirect" href="/wiki/Long_Island,_New_York" title="Long Island, New York">Long Island, New York</a>, United States</td>
<td><a class="mw-redirect" href="/wiki/Long_Island,_New_York" title="Long Island, New York">Long Island, New York</a>, United States</td>
Origin: Long Island, New York, United States
Genre: Hip hop
is_band=0
